# Importando as Bibliotecas

In [840]:
# import pandas
import pandas as pd
import seaborn as sn

In [841]:
import spacy

In [842]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# from imblearn.over_sampling import SMOTE

In [843]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [844]:
import re

# Conhecendo os dados

In [845]:
df = pd.read_csv(filepath_or_buffer="data/Dataset_5971.csv", sep=",")

In [846]:
display(df)

,LABEL,TEXT,URL,EMAIL,PHONE
0,ham,Your opinion about me? 1. Over 2. Jada 3. Kusr...,No,No,No
1,ham,What's up? Do you want me to come online? If y...,No,No,No
2,ham,So u workin overtime nigpun?,No,No,No
3,ham,"Also sir, i sent you an email about how to log...",No,No,No
4,Smishing,Please Stay At Home. To encourage the notion o...,No,No,No
...,...,...,...,...,...
5965,ham,:( but your not here....,No,No,No
5966,ham,Becoz its &lt;#&gt; jan whn al the post ofic...,No,No,No
5967,ham,Its a valentine game. . . send dis msg to all ...,No,No,No
5968,ham,We r outside already.,No,No,No


In [847]:
df.dtypes

LABEL    object
TEXT     object
URL      object
EMAIL    object
PHONE    object
dtype: object

In [848]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5970 entries, 0 to 5969
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   LABEL   5970 non-null   object
 1   TEXT    5970 non-null   object
 2   URL     5970 non-null   object
 3   EMAIL   5970 non-null   object
 4   PHONE   5970 non-null   object
dtypes: object(5)
memory usage: 233.3+ KB


In [849]:
df.describe()

,LABEL,TEXT,URL,EMAIL,PHONE
count,5970,5970,5970,5970,5970
unique,5,5949,2,2,2
top,ham,FREE MESSAGE Activate your 500 FREE Text Messa...,No,No,No
freq,4843,2,5763,5951,5126


In [850]:
df["LABEL"].unique()

array(['ham', 'Smishing', 'spam', 'Spam', 'smishing'], dtype=object)

In [851]:
df["URL"].unique()

array(['No', 'yes'], dtype=object)

In [852]:
df["EMAIL"].unique()

array(['No', 'yes'], dtype=object)

In [853]:
df["PHONE"].unique()

array(['No', 'yes'], dtype=object)

In [854]:
len([x for x in df.duplicated() if x == True])

17

In [855]:
i = 0
index_dup = []
for x in df.duplicated():
    if x == True:
        index_dup.append(i)
    i += 1

In [856]:
df.iloc[index_dup]

,LABEL,TEXT,URL,EMAIL,PHONE
1351,Smishing,UR GOING 2 BAHAMAS! CallFREEFONE 08081560665 a...,No,No,yes
1803,Smishing,Someone U know has asked our dating service 2 ...,No,No,yes
2962,ham,He remains a bro amongst bros,No,No,No
3691,ham,Reverse is cheating. That is not mathematics.,No,No,No
3873,ham,The sign of maturity is not when we start sayi...,No,No,No
3907,Smishing,U have a secret admirer. REVEAL who thinks U R...,No,No,yes
3908,Smishing,Congrats! 2 mobile 3G Videophones R yours. cal...,No,No,yes
4141,ham,The greatest test of courage on earth is to be...,No,No,No
4730,ham,"Sorry,in meeting I'll call later",No,No,No
4926,Smishing,URGENT!! Your 4* Costa Del Sol Holiday or £500...,No,No,yes


In [857]:
df[df["TEXT"] == "Sorry,in meeting I'll call later"]

,LABEL,TEXT,URL,EMAIL,PHONE
416,ham,"Sorry,in meeting I'll call later",No,No,No
4730,ham,"Sorry,in meeting I'll call later",No,No,No


In [858]:
df[df["TEXT"] == "Arun can u transfr me d amt"]

,LABEL,TEXT,URL,EMAIL,PHONE
414,ham,Arun can u transfr me d amt,No,No,No
5342,ham,Arun can u transfr me d amt,No,No,No


# Transformando os dados

In [859]:
df_transformado = df.copy()

In [860]:
df_transformado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5970 entries, 0 to 5969
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   LABEL   5970 non-null   object
 1   TEXT    5970 non-null   object
 2   URL     5970 non-null   object
 3   EMAIL   5970 non-null   object
 4   PHONE   5970 non-null   object
dtypes: object(5)
memory usage: 233.3+ KB


In [861]:
df_transformado["LABEL"].unique()

array(['ham', 'Smishing', 'spam', 'Spam', 'smishing'], dtype=object)

In [862]:
df_transformado["LABEL"].replace("Spam", "spam", inplace=True)
df_transformado["LABEL"].replace("Smishing", "smishing", inplace=True)
df_transformado["LABEL"].replace(["ham", "spam", "smishing"], [-1, 0, 1], inplace=True)

In [863]:
df_transformado["URL"].replace(["No", "yes"], [0, 1], inplace=True)
df_transformado["EMAIL"].replace(["No", "yes"], [0, 1], inplace=True)
df_transformado["PHONE"].replace(["No", "yes"], [0, 1], inplace=True)

In [864]:
display(df_transformado["URL"].unique())
display(df_transformado["EMAIL"].unique())
display(df_transformado["PHONE"].unique())

array([0, 1])

array([0, 1])

array([0, 1])

In [865]:
df_transformado.drop(["URL", "EMAIL", "PHONE"], axis=1, inplace=True)

In [866]:
len([x for x in df_transformado.duplicated() if x == True]) ## Ele aumenta por causa do tratamento que se fez para as palvras minusculas

20

In [867]:
df_transformado.drop_duplicates(inplace=True)

In [868]:
display(len(df_transformado))
display(len(df))

display(len(df) - len(df_transformado))

5950

5970

20

In [869]:
df_transformado.reset_index(drop=True, inplace=True)

In [870]:
display(df.head())
display(df_transformado.head())

,LABEL,TEXT,URL,EMAIL,PHONE
0,ham,Your opinion about me? 1. Over 2. Jada 3. Kusr...,No,No,No
1,ham,What's up? Do you want me to come online? If y...,No,No,No
2,ham,So u workin overtime nigpun?,No,No,No
3,ham,"Also sir, i sent you an email about how to log...",No,No,No
4,Smishing,Please Stay At Home. To encourage the notion o...,No,No,No


,LABEL,TEXT
0,-1,Your opinion about me? 1. Over 2. Jada 3. Kusr...
1,-1,What's up? Do you want me to come online? If y...
2,-1,So u workin overtime nigpun?
3,-1,"Also sir, i sent you an email about how to log..."
4,1,Please Stay At Home. To encourage the notion o...


# Analisando os dados

In [871]:
display(len(df_transformado[df_transformado["LABEL"] == -1]))
display(len(df_transformado[df_transformado["LABEL"] == 0]))
display(len(df_transformado[df_transformado["LABEL"] == 1]))

4833

489

628

# NLP

#### 1500 para treino e o resto para validação

In [872]:
# !python -m spacy download en_core_web_trf

In [873]:
nlp = spacy.load("en_core_web_trf")

In [874]:
def remover_stopwords(text):
    doc = nlp(text)
    frases_sem_stopwords = " ".join([token.text for token in doc if not token.is_stop])
    return frases_sem_stopwords

In [875]:
def remover_espacos(text):
    frases_sem_espaco_frente = " ".join(text.split())
    return frases_sem_espaco_frente

In [876]:
def remover_caractere_especial(text):
    texto_sem_cs = re.sub(r"ï¿½", r"", text)
    texto_sem_cs = re.sub(r"&lt;([^>]+)&gt;", r"", texto_sem_cs)
    texto_sem_cs = re.sub(r"�", r"", texto_sem_cs)
    texto_sem_cs = re.sub(r"ü", r"u", texto_sem_cs)
    texto_sem_cs = re.sub(r"Ü", r"U", texto_sem_cs)
    return texto_sem_cs

In [877]:
def remover_pontos(text):
    doc = nlp(text)
    frases_sem_ponto = " ".join([token.text for token in doc if not (token.is_punct and not re.match(r"(:|;)(-){0,1}(\)|\()", token.text))])
    return frases_sem_ponto

In [878]:
def lemmatizar(text):
    doc = nlp(text)
    frases_lematizadas = " ".join([token.lemma_ for token in doc])
    return frases_lematizadas

In [879]:
def pre_processamento(text):
    texto_processado = text
    print(texto_processado)
    texto_processado = lemmatizar(texto_processado)
    texto_processado = remover_caractere_especial(texto_processado)
    texto_processado = remover_espacos(texto_processado)  
    texto_processado = remover_pontos(texto_processado)
    # texto_processado = remover_stopwords(texto_processado)
    print(texto_processado)
    return texto_processado

In [880]:
df_transformado["PROCESSED_TEXT"] = df_transformado["TEXT"].apply(pre_processamento)

Your opinion about me? 1. Over 2. Jada 3. Kusruthi 4. Lovable 5. Silent 6. Spl character 7. Not matured 8. Stylish 9. Simple Pls reply..
your opinion about I 1 over 2 Jada 3 Kusruthi 4 lovable 5 silent 6 spl character 7 not matured 8 stylish 9 simple pl reply
What's up? Do you want me to come online? If you are free we can talk sometime�
what be up do you want I to come online if you be free we can talk sometime
So u workin overtime nigpun?
so u workin overtime nigpun
Also sir, i sent you an email about how to log into the usc payment portal. I.ll send you another message that should explain how things are back home. Have a great weekend.
also sir I send you an email about how to log into the usc payment portal i.ll send you another message that should explain how thing be back home have a great weekend
Please Stay At Home. To encourage the notion of staying at home. All tax-paying citizens are entitled to �305.96 or more emergency refund. smsg.io/fCVbD
please stay at home to encourage

In [881]:
i = 0
for x in df_transformado["PROCESSED_TEXT"].duplicated():
    if x == True:
        print([df_transformado["LABEL"].iloc[i]])
        print(i, [df_transformado["PROCESSED_TEXT"].iloc[i]])
    i += 1

[1]
320 ['do you want a new video handset 750 any time any network min unlimited text camcorder reply or call now 08000930705 for del Sat am']
[1]
431 ['please call 08712402779 immediately as there be an urgent message wait for you']
[1]
869 ['09066362231 urgent your mobile no 07xxxxxxxxx win a £ 2,000 bonus caller prize on 02/06/03 this be the 2nd attempt to reach you call 09066362231 asap']
[0]
928 ['free top ringtone -sub to weekly ringtone get 1st week free send subpoly to 81618-?3 per week stop sms-08718727870']
[1]
990 ['free MESSAGE activate your 500 free text message by reply to this message with the word free for term condition visit www.07781482378.com']
[1]
1034 ['the current lead bid be 151 to pause this auction send out customer care 08718726270']
[1]
1066 ['449050000301 you have win a £ 2,000 price to claim call 09050000301']
[0]
1212 ['free entry into our £ 250 weekly comp just send the word enter to 84128 now 18 t&c www.textcomp.com cust care 08712405020']
[1]
1364 ['yo

In [882]:
i = 0
for x in df_transformado["PROCESSED_TEXT"] == "":
    if x == True:
        print([df_transformado["LABEL"].iloc[i]])
        print(i, [df_transformado["PROCESSED_TEXT"].iloc[i]])
    i += 1

In [883]:
df_transformado.drop_duplicates(subset=["LABEL","PROCESSED_TEXT"], inplace=True)

In [884]:
#Remover os que estão duplicados
#TODO: conferir se foram os melhores dados
df_transformado.drop_duplicates(subset=["PROCESSED_TEXT"], inplace=True)

In [885]:
df_transformado.head(20)

,LABEL,TEXT,PROCESSED_TEXT
0,-1,Your opinion about me? 1. Over 2. Jada 3. Kusr...,your opinion about I 1 over 2 Jada 3 Kusruthi ...
1,-1,What's up? Do you want me to come online? If y...,what be up do you want I to come online if you...
2,-1,So u workin overtime nigpun?,so u workin overtime nigpun
3,-1,"Also sir, i sent you an email about how to log...",also sir I send you an email about how to log ...
4,1,Please Stay At Home. To encourage the notion o...,please stay at home to encourage the notion of...
5,1,BankOfAmerica Alert 137943. Please follow http...,BankOfAmerica alert 137943 please follow http:...
6,-1,Sorry dude. Dont know how i forgot. Even after...,sorry dude do nt know how I forget even after ...
7,-1,I don't quite know what to do. I still can't g...,I do n't quite know what to do I still ca n't ...
8,-1,Ok lor. Anyway i thk we cant get tickets now c...,ok lor anyway I thk we ca nt get ticket now co...
9,-1,Wat r u doing now?,wat r u do now


In [886]:
# TODO: Melhorar alguns lemmas, colocar stemm, retirar os caracteres especiais
display(df_transformado["PROCESSED_TEXT"].iloc[1])
display(df_transformado["TEXT"].iloc[1])

'what be up do you want I to come online if you be free we can talk sometime'

"What's up? Do you want me to come online? If you are free we can talk sometime�"

In [887]:
display(len(df_transformado[df_transformado["LABEL"] == -1]))
display(len(df_transformado[df_transformado["LABEL"] == 0]))
display(len(df_transformado[df_transformado["LABEL"] == 1]))

4828

458

554

# Treinamento

In [888]:
df_teste = df_transformado.iloc[:1500].copy()
df_validacao = df_transformado.iloc[1500:].copy()

In [889]:
x = df_teste["PROCESSED_TEXT"]
y = df_teste["LABEL"]

In [890]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)

In [891]:
class_weights = {-1: 1, 0: 2, 1: 4}

In [892]:
vectorizer = TfidfVectorizer()
x_train_tfidf = vectorizer.fit_transform(x_train)
x_test_tfidf = vectorizer.transform(x_test)

##### Logistic Regression

In [893]:
model = LogisticRegression(solver="lbfgs", multi_class="multinomial", random_state=42, class_weight=class_weights)
model.fit(x_train_tfidf, y_train)

LogisticRegression(class_weight={-1: 1, 0: 2, 1: 4}, multi_class='multinomial',
                   random_state=42)

In [894]:
y_pred = model.predict(x_test_tfidf)

#### Random Florest

In [895]:
rf_classifier = RandomForestClassifier(random_state=42, class_weight=class_weights)
rf_classifier.fit(x_train_tfidf, y_train)

RandomForestClassifier(class_weight={-1: 1, 0: 2, 1: 4}, random_state=42)

In [896]:
y_pred = rf_classifier.predict(x_test_tfidf)

#### SVM

In [897]:
svm_classifier = SVC(kernel='linear',random_state=42, class_weight=class_weights)
svm_classifier.fit(x_train_tfidf, y_train)

SVC(class_weight={-1: 1, 0: 2, 1: 4}, kernel='linear', random_state=42)

In [898]:
y_pred = svm_classifier.predict(x_test_tfidf)

# Teste

In [899]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")
print(f'Relatório de Classificação:\n{report}')

Acurácia do modelo: 0.94
Relatório de Classificação:
              precision    recall  f1-score   support

          -1       0.97      0.98      0.98       186
           0       0.70      0.54      0.61        13
           1       0.79      0.85      0.81        26

    accuracy                           0.94       225
   macro avg       0.82      0.79      0.80       225
weighted avg       0.94      0.94      0.94       225



# Validação

In [900]:
x_v = vectorizer.transform(df_validacao["PROCESSED_TEXT"])
y_v = df_validacao["LABEL"]
y_pred = model.predict(x_v)

In [901]:
accuracy = accuracy_score(y_v, y_pred)
report = classification_report(y_v, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")
print(f'Relatório de Classificação:\n{report}')

Acurácia do modelo: 0.94
Relatório de Classificação:
              precision    recall  f1-score   support

          -1       0.96      0.99      0.98      3610
           0       0.82      0.54      0.65       335
           1       0.81      0.79      0.80       395

    accuracy                           0.94      4340
   macro avg       0.86      0.77      0.81      4340
weighted avg       0.94      0.94      0.94      4340

